In [1]:
include("../src/sys_operators.jl")
include("../src/pimc_cluster_samp.jl")
include("../src/clusterization.jl")

using .Clusterization
using .PIMC
using .SysOperators
using LinearAlgebra
using Plots
using Printf
using SparseArrays

# NMM to test energy estimator

### N=2

In [ ]:
N = 2
m_max = 5
g = 1

Nstates = 2*m_max+1
Id = id_op(Nstates)
ops = RotorOps(m_max = m_max, g = g)
Ki = ops.M2
Vij = ops.Vij
# expVij = abs.(round.(exp(-tau * Vij), digits=14))
# # expVij = abs.(exp(-tau * Vij))

K1 = kron(Ki,Id)
K2 = kron(Id,Ki)

K = K1 + K2

V = Vij

In [ ]:
T = 0.1
L = 300

beta = 1.0/T
tau = beta/L
num_total_states = Nstates^N

expK = abs.(round.(exp(-tau * K), digits=14))

expV = abs.(round.(exp(-tau * V), digits=14))

In [ ]:
Psi_T = [1,1]*(m_max+1)
m0 = map_vector2index(Nstates,Psi_T)

rho = ((expK * expV)^(L-1)) * expK

rho_aux = rho * expV

Z0 = rho_aux[m0,m0]

In [ ]:
Z = 0.0
E = 0.0
aux = 0
for p = 1:num_total_states
    zp = expV[p,m0]
    ep = 0
    for a = 1:num_total_states
        ep += expV[p,a] * V[a,m0]
    end
    if zp==0
        println(ep)
    end
    Z += rho[m0,p] * zp
    E += rho[m0,p] * ep
end

println(E,"    ",Z,"    ",E/Z)

In [ ]:
H = K + V
F = eigen(H)
F.values[1]

# N=3 rotors

In [4]:
N = 3
m_max = 5
g = 1
Nstates = 2*m_max+1
Id = id_op(Nstates)
ops = RotorOps(m_max = m_max, g = g)
K = ops.M2
Vij = ops.Vij
# expVij = abs.(round.(exp(-tau * Vij), digits=14))
# # expVij = abs.(exp(-tau * Vij))

K1 = kron(K,Id,Id)
K2 = kron(Id,K,Id)
K3 = kron(Id,Id,K)

K_total = K1 + K2 + K3

V12 = kron(Vij,Id)
V23 = kron(Id,Vij)
V_total = (V12 + V23)

H_total = K_total + V_total

1331×1331 Matrix{Float64}:
 75.0   0.0   0.0    0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
  0.0  66.0   0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   0.0  59.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   0.0   0.0   54.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   0.0   0.0    0.0   51.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   0.0   0.0    0.0    0.0   50.0   …   0.0    0.0    0.0    0.0    0.0
  0.0   0.0   0.0    0.0    0.0    0.0      -0.75   0.0    0.0    0.0    0.0
  0.0   0.0   0.0    0.0    0.0    0.0       0.0   -0.75   0.0    0.0    0.0
  0.0   0.0   0.0    0.0    0.0    0.0       0.0    0.0   -0.75   0.0    0.0
  0.0   0.0   0.0    0.0    0.0    0.0       0.0    0.0    0.0   -0.75   0.0
  ⋮                                ⋮     ⋱                               ⋮
  0.0   0.0  -0.75   0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   0.0   0.0   -0.75   0.0    0.0       0.0   

In [22]:
F = eigen(H_total)
ket_E0 = F.vectors[:,1]

ket_E0'K_total'ket_E0

0.9475909313013755

# N=2 Rotors

In [25]:
N = 2
m_max = 5
g = 1
Nstates = 2 * m_max + 1
Id = id_op(Nstates)
ops = RotorOps(m_max=m_max, g=g)
K = ops.M2
Vij = ops.Vij
# expVij = abs.(round.(exp(-tau * Vij), digits=14))
# # expVij = abs.(exp(-tau * Vij))

K1 = kron(K, Id)
K2 = kron(Id, K)

K_total = K1 + K2

V12 = Vij
V_total = V12

H_total = K_total + V_total

121×121 Matrix{Float64}:
 50.0    0.0    0.0    0.0    0.0   …   0.0    0.0    0.0    0.0   -0.75
  0.0   41.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0   34.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0   29.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0   26.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0   …  -0.25   0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0   -0.25   0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0      -0.75   0.0   -0.25   0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0   -0.75   0.0   -0.25   0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0   -0.75   0.0   -0.25
  ⋮                                 ⋱                               ⋮
  0.0   -0.25   0.0   -0.75   0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0   -0.25   0.0   -0.75      0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0   -0.25   0.0      

In [28]:
F = eigen(H_total)
ket_E0 = F.vectors[:, 1]

ket_E0'K_total'ket_E0

0.3941384464777084

In [ ]:
T = 0.1
L = 100

beta = 1.0/T
tau = beta/L
Nstates = 2*m_max+1
num_total_states = Nstates^N

# expK = abs.(round.(exp(-tau * K_total), digits=14))

# expV12 = abs.(round.(exp(-tau * V12), digits=14))
# expV23 = abs.(round.(exp(-tau * V23), digits=14))

expK = abs.(exp(-tau * K_total))

expV12 = abs.(exp(-tau * V12))
expV23 = abs.(exp(-tau * V23))

In [ ]:
Psi_T = [1,1,1]*(m_max+1)
m0 = map_vector2index(Nstates,Psi_T)

rho = ((expK * expV12 * expV23)^(L-1)) * expK * expV12

rho_aux = rho * expV23

Z0 = rho_aux[m0,m0]

In [ ]:
Z = 0.0
E = 0.0
for p = 1:num_total_states

    zp = expV23[p,m0]
    ep = 0
    for a = 1:num_total_states
        ep += (expV23[p,a] * V_total[a,m0])
    end
    if (zp == 0) || (zp < 1e-14)
        state = map_index2vector(Nstates, N, p) .- (m_max + 1)
        if (ep != 0.0) || (ep > 1e-14)
            println("term avoided: ", state,"    energy contribution: ",ep)
        end
    end
    Z += rho[m0,p] * zp
    E += rho[m0,p] * ep
end
println(E,"    ",Z,"    ",E/Z)

In [ ]:
e = 0.0
for p = 1:num_total_states
    for a = 1:num_total_states
        e += rho[m0,p] * expV23[p,a] * V_total[a,m0]
    end
end
e = e/Z0

In [ ]:
oper = rho_aux * V_total
E = oper[m0,m0]
E = E/Z0

# Comparing end beads contribution

In [2]:
N = 3
m_max = 5
g = 1
Nstates = 2*m_max+1
num_total_states = Nstates^N

T = 0.1
L = 100

beta = 1.0/T
tau = beta/L

Id = id_op(Nstates)
ops = RotorOps(m_max = m_max, g = g)
K = ops.M2
Vij = ops.Vij

K1 = kron(K,Id,Id)
K2 = kron(Id,K,Id)
K3 = kron(Id,Id,K)

K_total = K1 + K2 + K3

V12 = kron(Vij,Id)
V23 = kron(Id,Vij)
V_total = V12 + V23

H_total = K_total + V_total

# expH = round.(abs.(exp(-tau * K_total)), digits=14)
expH = abs.(exp(-tau * H_total))
expH_H = expH * H_total

1331×1331 Matrix{Float64}:
  0.0419972     1.02125e-17   0.000131683  …   7.56948e-6   -2.06854e-17
  1.27946e-17   0.0907642     1.11751e-16     -3.389e-16     7.56948e-6
  0.000131683   1.29276e-16   0.163254         2.37889e-9   -3.70375e-16
 -2.77783e-15   0.000219996   4.36841e-17     -1.89099e-12   2.44972e-9
  4.60314e-8   -2.2267e-15    0.000311336     -7.48443e-13  -1.26109e-12
 -3.06405e-12   5.31451e-8   -1.93436e-15  …   6.88282e-10  -1.18693e-12
 -7.23549e-12  -9.90263e-13   5.41626e-8      -1.01428e-15   1.44358e-9
  3.70396e-8   -1.24424e-11  -1.58569e-13      1.26391e-5   -8.12035e-16
 -3.39106e-15   3.22785e-8   -1.46971e-11     -2.60384e-16   7.97882e-6
  8.45584e-5   -3.83321e-15   3.22737e-8       0.00575783   -1.15669e-16
  ⋮                                        ⋱                 ⋮
  7.97882e-6   -2.25432e-16   0.0101372        3.22785e-8   -3.42911e-15
 -8.39449e-16   1.26391e-5   -4.61254e-16     -1.24424e-11   3.70396e-8
  1.44358e-9   -9.39223e-16   1.69431e-

In [13]:
Psi_T = [1,1,1]*(m_max+1)
m0 = map_vector2index(Nstates,Psi_T)

for p = 1:num_total_states
    zp = expH[p,m0]
    ep = expH_H[p,m0]
    if (zp == 0) || (abs(zp) < 1e-14)
        if (abs(ep) > 1e-13)
            state = map_index2vector(Nstates, N, p) .- (m_max + 1)
            println("term avoided: ", state,"    energy contribution: ",ep)
        end
    end
end

term avoided: [-5, -5, 4]    energy contribution: -1.9477281484604913e-13
term avoided: [-5, -3, -3]    energy contribution: -1.9683824269462295e-13
term avoided: [-5, -1, -4]    energy contribution: -6.124698605547695e-13
term avoided: [-5, -1, -3]    energy contribution: -6.572754824729104e-13
term avoided: [-5, -1, 3]    energy contribution: -1.3842730369646084e-13
term avoided: [-5, 1, -4]    energy contribution: -2.0428662623766601e-13
term avoided: [-5, 1, 3]    energy contribution: -4.152319182455468e-13
term avoided: [-5, 2, -4]    energy contribution: -1.6927705507790908e-13
term avoided: [-5, 3, 4]    energy contribution: -5.11386431582661e-13
term avoided: [-5, 4, -4]    energy contribution: -4.505510059211466e-13
term avoided: [-5, 4, -3]    energy contribution: -5.959348562616188e-13
term avoided: [-4, -4, -3]    energy contribution: -3.0773049133037617e-13
term avoided: [-4, -3, 2]    energy contribution: -4.640159791578742e-13
term avoided: [-4, -3, 4]    energy contribu

In [5]:
L = 50
tau = beta / L

rho_trotter1 = ((exp_matrix(-tau * K) * exp_matrix(-tau * V))^L)
rho_trotter2 = (exp_matrix(-0.5*tau*V) * exp_matrix(-tau*K) * exp_matrix(-0.5*tau*V))^L

E0_trotter1 = ((rho_trotter1*H)[idx0, idx0]) / (rho_trotter1[idx0, idx0])
E0_trotter2 = ((rho_trotter2*H)[idx0, idx0]) / (rho_trotter2[idx0, idx0])

println(tau)
# println(E0_trotter1)
println(E0_trotter2)
println("oi")

0.2
-1.1902269552126148
oi


In [ ]:
dig = 12
x = round.(abs.(exp(-tau * V12)) * abs.(exp(-tau * V23)), digits=dig)
y = round.(abs.(exp(-tau * (V12+V23))), digits=dig)

x == y

In [ ]:
V12*V23 == V23*V12

In [ ]:
2-1.18

In [ ]:
for i=1:1331
    for j=1:1331
        if expH[i,j] < 0.1
            println(expH[i,j])
        end
    end
end

In [ ]:
if 1 > 2 && 4>3
    print("oi")
end

In [ ]:
print(N)

In [ ]:
flush(stdout)

In [ ]:
N = 3
m_max = 5
g = 1
T = 0.1
L = 50

beta = 1.0/T
tau = beta/L
Nstates = 2*m_max+1

if (N == 2)		
    P = L +1
elseif (N > 2)
    P = 2*L + 1        
end
tau

In [ ]:
ops = RotorOps(m_max = m_max, g = g)
Vij = ops.Vij
expVij = abs.(round.(exp(-tau * Vij), digits=14))
# expVij = abs.(exp(-tau * Vij))

In [12]:
280/12

23.333333333333332

In [ ]:
# cluster = Dict{Float64,Vector{Int64}}()
# for m1 = 1:Ngrid
#     for m2 = 1:Ngrid
#         for m1p = 1:Ngrid
#             for m2p = 1:Ngrid
# 				idx = map_vector2index(PI.Nstates, [m1,m2, m1p, m2p])
#                 m_el = mel([m1, m2], rho, [m1p, m2p])

# 				if m_el != 0
#                     if m_el in keys(cluster)
#                         append!(cluster[m_el], idx)
# 					else
#                         push!(cluster, m_el => [idx])
# 					end

# 				end
				
#             end
#         end
#     end
# end
# cluster = sort(cluster)
# cluster_new = Dict{Tuple{String,Float64},Vector{Int64}}()
# count = 1
# for key in keys(cluster)
# 	key_new = tuple(string("C",count),key)
# 	println(key_new)
#     push!(cluster_new, key_new  => cluster[key])
# 	count += 1
# end

cluster = Dict{Float64, Vector{Vector{Int64}}}()
for m1 = 1:Nstates
    for m2 = 1:Nstates
        for m1p = 1:Nstates
            for m2p = 1:Nstates
				idx = map_vector2index(Nstates, [m1,m2, m1p, m2p])
                m_el = mel([m1, m2], expVij, [m1p, m2p])
                M = [m1,m2, m1p, m2p] .- (m_max+1)
				if m_el != 0
                    if m_el in keys(cluster)
                        append!(cluster[m_el], [M])
					else
                        push!(cluster, m_el => [M])
					end

				end
				
            end
        end
    end
end
cluster = sort(cluster)

In [ ]:
open("out_cluster.txt", "w") do file
    count = 1
    for key in keys(cluster)
        space = repeat(" ",20-length(string(key)))
        ans = string(key, space, cluster[key][:], "\n")
        write(file, ans)
		count += 1
    end
end

In [ ]:
cluster_int = Dict{Float64,Vector{Tuple{Int64,Int64}}}()
for m1 = 1:Ngrid
    for m2 = 1:Ngrid
        for m1p = 1:Ngrid
            for m2p = 1:Ngrid
                for m3p = 1:Ngrid
                    for m2pp = 1:Ngrid
                        for m3pp = 1:Ngrid
                            m_el = round(
                                mel([m1, m2], rho, [m1p, m2p]) *
                                mel([m2p, m3p], rho, [m2pp, m3pp]), digits=15)
                            if m_el != 0
                                idx1 = map_vector2index(Ngrid, [m1, m2, m1p, m2p])
                                idx2 = map_vector2index(Ngrid, [m2p, m3p, m2pp, m3pp])
                                if m_el in keys(cluster_int)
                                    append!(cluster_int[m_el], [(idx1,idx2)])
								else
                                    push!(cluster_int, m_el => [(idx1,idx2)])
								end
                                
                            end
                        end
                    end
                end
            end
        end
    end
end
# cluster_int = sort(cluster_int)

In [ ]:
open("out_cluster.txt", "w") do file
    count = 1
    for key in keys(cluster)
        for index in 
        particles = map_index2vector(Nstates, 4, index)
        ans = string(key, "   P",count,":", cluster[key], "\n")
        write(file, ans)
		count += 1
    end
end

In [ ]:
dim = length(cluster)
vec =[]
for keyi in keys(cluster)
	for keyj in keys(cluster)
		prod = round(keyi * keyj, digits=14)
		if (prod ∉ vec)
			append!(vec, [prod])
		end
	end
end
vec_sorted = sort(vec)
# for i=1:length(vec)
# println(i,"  ", vec[i])
# end

open("out_cluster_prods.txt", "w") do file
	for key in vec_sorted
		ans = string(key, "\n")
		write(file, ans)
	end
end

In [ ]:
open("out_cluster_int.txt", "w") do file
    for key in keys(cluster_int)
        ans = string(key, "    ", cluster_int[key], "\n")
        write(file, ans)
    end    
end

In [ ]:
map_index2vector(Ngrid, 4, 14397)

In [ ]:
N = 3
m_max = 5
Nstates = 2*m_max+1
num_total_states = Nstates^N
for index = 1:num_total_states
    Malpha = map_index2vector(Nstates, N, index)
    println(Malpha[:])
end


### Testing how sparse the density matrices are

In [ ]:
MC_steps = 1000
N = 2
g = 10
T = 0.1
L = 45

for m = 1:15
	PI = PathIntegral(N=N, m_max=m, g=g, L=L, MC_steps=MC_steps, T=T)
	ops = RotorOps(m_max=PI.m_max, g=PI.g)
	Vij = ops.Vij
	Kij = ops.Kij
	Id = id_op(PI.Nstates)
	Hij = Vij + Kij
	# expVij = exp(-PI.tau * Vij)
	# expKij = exp(-0.5 * PI.tau * Kij)
	# rho = round.(expKij * expVij * expKij, digits=14)
    rho = abs.(round.(exp(-PI.tau * Hij), digits=14))
	Ngrid = PI.Nstates

	count = 0
	for m1 = 1:Ngrid
		for m2 = 1:Ngrid
			for m1p = 1:Ngrid
				for m2p = 1:Ngrid
                    idx = map_vector2index(PI.Nstates, [m1, m2, m1p, m2p])
					m_el = mel([m1, m2], rho, [m1p, m2p])
					if m_el < 0
						println(m_el)
						count += 1
					end
				end
			end
		end
	end

	println("m_max=",m,"   non-zero matrix elements:  ",round((count*100)/(Ngrid^4), digits = 2),"%")

end

In [ ]:
g = 1

for m = 1:15
	PI = PathIntegral(N=N, m_max=m, g=g, L=L, MC_steps=MC_steps, T=T)
	ops = RotorOps(m_max=PI.m_max, g=PI.g)
	Vij = ops.Vij
	Kij = ops.Kij
	Id = id_op(PI.Nstates)
	Hij = Vij + Kij
	# expVij = exp(-PI.tau * Vij)
	# expKij = exp(-0.5 * PI.tau * Kij)
	# rho = round.(expKij * expVij * expKij, digits=14)
    rho = abs.(round.(exp(-PI.tau * Hij), digits=14))
	Ngrid = PI.Nstates

	count = 0
	for m1 = 1:Ngrid
		for m2 = 1:Ngrid
			for m1p = 1:Ngrid
				for m2p = 1:Ngrid
                    idx = map_vector2index(PI.Nstates, [m1, m2, m1p, m2p])
					m_el = mel([m1, m2], rho, [m1p, m2p])
					if m_el < 0
						println(m_el)
						count += 1
					end
				end
			end
		end
	end

	println("m_max=",m,"   non-zero matrix elements:  ",round((count*100)/(Ngrid^4), digits = 2),"%")

end

In [ ]:
m_max = 5
Ngrid = 2*m_max+1
g=1

ops = RotorOps(m_max=m_max, g=g)
Vij = ops.Vij
K = ops.M2
Id = id_op(ops.Ngrid)


K1 = kron(K, id_op(Ngrid), id_op(Ngrid))
K2 = kron(id_op(Ngrid), K, id_op(Ngrid))
K3 = kron(id_op(Ngrid), id_op(Ngrid), K)

V12 = kron(Vij, id_op(Ngrid))
V23 = kron(id_op(Ngrid), Vij)

K_total = K1+K2+K3
V_total = V12+V23

H = K_total + V_total
# F = eigen(H)
# F.values[1]

In [ ]:
Psi_T = [1,1,1]*(m_max+1)
idx = map_vector2index(Ngrid,Psi_T)

T=0.0005

beta = 1/T

# E0 = []
# E0_trotter = []
# tau_list = []

rho = exp(-beta*H)
est = (rho*H)[idx,idx]/rho[idx,idx]

# for l = 10:10:100

#     tau = beta/l
#     expK = exp(-tau*K_total)
#     expV = exp(-tau*V_total)
#     rho_trotter = (expK*expV)^l

#     est_trotter = (rho_trotter*H)[idx,idx]/rho_trotter[idx,idx]
#     append!(tau_list, tau)
#     append!(E0, est)
#     append!(E0_trotter, est_trotter)
# end

In [ ]:
using PyPlot
plt.clf()
plt.plot(tau_list, E0, color="blue", linewidth=2.0, linestyle="--")
plt.plot(tau_list, E0_trotter, color="red", linewidth=2.0, linestyle="--")
gcf()

In [ ]:
N = 3
m_max = 5
g = 1.0
T = 0.1
L = 1000
tau = 1/(T*L)

Nstates = 2*m_max+1

ops = RotorOps(m_max = m_max, g = g)
Vij = ops.Vij
expVij = (exp_matrix(-tau * Vij))

m0 = m_max+1
m0_vec = ones(Int64, N) * (m0-4)


num_total_states = Nstates^N
E0 = 0

for index = 1:num_total_states
    Malpha = map_index2vector(Nstates, N, index)
                        
    rho = 1
    gamma = 0
    for i=1:N-1
        remain_Malpha = append!(Malpha[1:i-1],Malpha[i+2:N])
        remain_path = append!(m0_vec[1:i-1],m0_vec[i+2:N])

        if remain_Malpha == remain_path
            if (i%2==0)
                rho *= (mel(m0_vec[i:i+1], expVij, Malpha[i:i+1])
                /
                mel(m0_vec[i:i+1], expVij, m0_vec[i:i+1])
            )
            end
            gamma += mel(Malpha[i:i+1], Vij, m0_vec[i:i+1]) 								                          	                            
        end
    end
    E0 += rho*gamma				
end

print(E0)

In [ ]:
wait_for_key(prompt) = (print(stdout, prompt); read(stdin, 1); nothing)

In [ ]:
N = 3
m_max = 5
g = 1.0
T = 0.1
L = 10
tau = 1/(T*L)

Nstates = 2*m_max+1

ops = RotorOps(m_max = m_max, g = g)
Vij = ops.Vij
expVij = (exp_matrix(-tau * Vij))
for mi = 1:Nstates
    for mj = 1:Nstates
        for mk = 1:Nstates
            for ml = 1:Nstates                       
                mel_expVij = mel([mi,mj], expVij, [mk,ml])

                idx = map_vector2index(PI.Nstates, [mi,mj,mk,ml])
            end
        end
    end
end

In [ ]:
V = zeros(Int64, (5, 5))

In [ ]:
V[1,3] = V[3,1] = V[5,3] = V[3,5] = 3
V[2,3] = V[3,2] = V[3,4] = V[4,3] = 1

In [ ]:
F = eigen(V)

In [ ]:
F.values

In [ ]:
N = 3
m_max = 5
g = 1.0
Nstates = 2*m_max+1

m0 = m_max + 1
idx0 = map_vector2index(Nstates, [m0, m0, m0])

ops = RotorOps(m_max=m_max, g=g)
Vij = ops.Vij
Ki = ops.M2
Id = id_op(Nstates)

K = kron(Ki, Id, Id) + kron(Id, Ki, Id) + kron(Id, Id, Ki)
V = kron(Vij, Id) + kron(Id, Vij)
H = K + V